Supp to `other_method_reg2` via adding AIC and BIC

Due to diff random seed, we can not exactly reproduce the results, 
so I use the results in `other_method_reg2`

In [1]:
import sys
sys.path.append("../../")
from pathlib import Path
root = Path("../../")

PosixPath('../..')

In [2]:
import importlib
from pyTVDN import TVDNDetect

import pyTVDN.utils
importlib.reload(pyTVDN.utils)
from pyTVDN.utils import load_pkl, save_pkl

from sklearn.cluster import KMeans
from pathlib import Path
from scipy.io import loadmat
import numpy as np
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
import os
from scipy import signal
from scipy.stats import fisher_exact, norm
import pickle
import seaborn as sns
#from tqdm.autonotebook import tqdm
from tqdm import tqdm
import numbers
import pandas as pd
%matplotlib inline

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score, roc_curve
from tmp_utils import get_ABIC

import warnings
warnings.filterwarnings('ignore')

### Load data

In [4]:
resDir = root/"results"
dataDir = root/"data"

PosixPath('../../data')

In [5]:
def wUFn(Amat):
    eigVals, eigVecs = np.linalg.eig(Amat)
    r = np.sum(np.cumsum(np.abs(eigVals))/np.sum(np.abs(eigVals)) <= 0.8) + 1
    if (eigVals[r-1].imag + eigVals[r].imag ) == 0:
        r = r + 1
    wU = np.abs(np.matmul(eigVecs[:, :r], eigVals[:r]))
    return wU, r, eigVals[:r]

In [6]:
def minmax(x):
    num = x - np.min(x)
    den = np.max(x) - np.min(x)
    return num/den

def U2BrainVec(wU):
    emVec = np.zeros_like(DKtmplate, dtype=np.float64)
    for idx in range(1, 69):
        emVec[DKtmplate==idx] = wU[idx-1]
    return emVec

def reorderU(wU):
    wUreorder = np.zeros_like(wU, dtype=np.float64)
    wUreorder[0::2] = wU[:34]
    wUreorder[1::2] = wU[34:]
    return wUreorder

In [7]:
def rSelFn(Amat):
    eigVals, eigVecs = np.linalg.eig(Amat)
    rSel = np.where(np.cumsum(np.abs(eigVals))/np.sum(np.abs(eigVals)) >0.8)[0][0] + 1
    # if breaking conjugate eigval pair, add r with 1
    if (eigVals[rSel-1].imag + eigVals[rSel].imag) == 0:
        rSel = rSel + 1
    wU = np.abs(np.matmul(eigVecs[:, :rSel], eigVals[:rSel]))
    return rSel, wU, eigVals[:rSel]

### Parameters

In [8]:
Cs = [0.001, 0.01, 0.05, 0.1, 0.2, 0.4, 0.8, 1, 1.6, 3.2, 6.4, 12.8, 25.6]
penalty = "l2"
freq = 120

120

### DMD

In [9]:
filName = f"Ctrl_data_NWM_DMD.pkl"
with open(resDir/filName, "rb") as f:
    CtrlDMD = pickle.load(f)
filName = f"AD_data_NWM_DMD.pkl"
with open(resDir/filName, "rb") as f:
    ADDMD = pickle.load(f)

In [10]:
fsAll = []
for v in ADDMD:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsAD = fsAll;

In [11]:
fsAll = []
for v in CtrlDMD:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsCtrl = fsAll;

In [12]:
fsName = ["r", "tQ", "tL", "wU"]

Ys = np.concatenate([np.ones(len(fsAD)), np.zeros(len(fsCtrl))])
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1)
Xs = np.concatenate([XsAD, XsCtrl], axis=0)
stdXs = (Xs - Xs.mean(axis=0))/Xs.std(axis=0);
    
select_fs_idx = np.array([ 0,  9, 16, 28, 40, 42, 45, 51, 60, 62, 65])
gOptC2 = 0.05

# select fs
stdXs_selected = stdXs[:, select_fs_idx]

get_ABIC(stdXs_selected, Ys, gOptC2)

(206.11451480637544, 221.3873379158028)

### ND(TV)

In [13]:
filName = f"Ctrl_data_NWM_TV.pkl"
with open(resDir/filName, "rb") as f:
    CtrlTV = pickle.load(f)
filName = f"AD_data_NWM_TV.pkl"
with open(resDir/filName, "rb") as f:
    ADTV = pickle.load(f)

In [14]:
fsAll = []
for v in ADTV:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsAD = fsAll;

In [15]:
fsAll = []
for v in CtrlTV:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsCtrl = fsAll;

In [16]:
fsName = ["r", "tQ", "tL", "wU"]
#fsName = ["r", "tQ", "tC", "tL", "wU"]
#fsName = ["r", "absImagEigValMaxMean", "tQ", "tC", "tL", "wU"]
Ys = np.concatenate([np.ones(len(fsAD)), np.zeros(len(fsCtrl))])
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1)
Xs = np.concatenate([XsAD, XsCtrl], axis=0)
stdXs = (Xs - Xs.mean(axis=0))/Xs.std(axis=0)


select_fs_idx = np.array([ 0,  4, 14, 20, 24, 34, 41, 43, 48, 52, 59, 62, 64, 69])
# select fs
stdXs_selected = stdXs[:, select_fs_idx]
stdXs_selected.shape

gOptC2 = 0.1

get_ABIC(stdXs_selected, Ys, gOptC2)

(211.60643764880226, 230.679199702888)

### Dynamic DMD

In [17]:
filName = f"AD_data_dynDMD_withgraph.pkl"
with open(resDir/filName, "rb") as f:
    fsAD = pickle.load(f)
    
filName = f"Ctrl_data_dynDMD_withgraph.pkl"
with open(resDir/filName, "rb") as f:
    fsCtrl = pickle.load(f)
    
# for abs imag max eig
filName = f"AD_data_dynDMD.pkl"
with open(resDir/filName, "rb") as f:
    fsAD_add = pickle.load(f)
    
filName = f"Ctrl_data_dynDMD.pkl"
with open(resDir/filName, "rb") as f:
    fsCtrl_add = pickle.load(f)
    
map_dict = ["tmC", "tmL", "tQ", "C", "L", "Q"]

['tmC', 'tmL', 'tQ', 'C', 'L', 'Q']

In [18]:
fs_TVDMD = fsAD
fs_TVDMD_add = fsAD_add

fsAll = []
for ix in range(len(fs_TVDMD)):
    raw_fs = edict(fs_TVDMD[ix])
    add_fs = edict(fs_TVDMD_add[ix])
    fs = edict()
    
    fs.r = raw_fs.r
    fs.ncpts = raw_fs.ncpts
    fs.absEigValMean = raw_fs.absEigValMean
    fs.wUmeanAll = raw_fs.wUmeanAll
    fs.absImagEigValMaxMean = add_fs.absImagEigValMaxMean
    
    fs.var_C = np.var([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.var_L = np.var([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.var_Q = np.var([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    fs.mean_C = np.mean([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.mean_L = np.mean([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.mean_Q = np.mean([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    dwell = np.diff(np.concatenate([[0], raw_fs.ecpts, [7200]])/freq)
    max_dwell_idx = np.argsort(-dwell)[0]
    fs.top1_C = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmC")]
    fs.top1_L = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmL")]
    fs.top1_Q = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tQ")]
    fs.max_dwell = np.max(dwell)
    
    
    fsAll.append(fs)
    
fsAD_trans = fsAll;

In [19]:
fs_TVDMD = fsCtrl
fs_TVDMD_add = fsCtrl_add

fsAll = []
for ix in range(len(fs_TVDMD)):
    raw_fs = edict(fs_TVDMD[ix])
    add_fs = edict(fs_TVDMD_add[ix])
    fs = edict()
    
    fs.r = raw_fs.r
    fs.ncpts = raw_fs.ncpts
    fs.absEigValMean = raw_fs.absEigValMean
    fs.wUmeanAll = raw_fs.wUmeanAll
    fs.absImagEigValMaxMean = add_fs.absImagEigValMaxMean
    
    fs.var_C = np.var([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.var_L = np.var([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.var_Q = np.var([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    fs.mean_C = np.mean([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.mean_L = np.mean([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.mean_Q = np.mean([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    dwell = np.diff(np.concatenate([[0], raw_fs.ecpts, [7200]])/freq)
    max_dwell_idx = np.argsort(-dwell)[0]
    fs.top1_C = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmC")]
    fs.top1_L = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmL")]
    fs.top1_Q = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tQ")]
    fs.max_dwell = np.max(dwell)
    
    
    fsAll.append(fs)
    
fsCtrl_trans = fsAll;

In [20]:
penalty = "l2"
Ys = np.concatenate([np.ones(len(fsAD_trans)), np.zeros(len(fsCtrl_trans))])
fsName = ["r", "ncpts",
          "mean_Q", "mean_L", 
          "var_Q",  "var_L",  
          "wUmeanAll" ]
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD_trans[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD_trans]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl_trans]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD_trans])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl_trans])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1)
Xs = np.concatenate([XsAD, XsCtrl], axis=0)
stdXs = (Xs - Xs.mean(axis=0))/Xs.std(axis=0)

select_fs_idx = np.array([0,  1,  4,  5,  9, 19, 27, 38, 45, 46, 48, 53, 54, 57, 61, 62, 64])
gOptC2 = 0.0005

stdXs_selected = stdXs[:, select_fs_idx]

get_ABIC(stdXs_selected, Ys, gOptC2)

(239.2570721376727, 239.8732254944296)